In [2]:
# 如果网络中有大量重复的结构，那么我们可以很紧凑的来构造这个网络，for example VGG
# VGG 的一个关键是使用很多相对小的 kernel(3x3) 的卷积层然后接一个池化层，之后将这个模块重复多次
from mxnet.gluon import nn

In [3]:
def vgg_block(num_convs, channels):
    net = nn.Sequential()
    for _ in range(num_convs):
        net.add(nn.Conv2D(channels=channels, kernel_size=3, padding=1, activation='relu'))
    net.add(nn.MaxPool2D(pool_size=2, strides=2))
    return net

In [4]:
from mxnet import nd
blk = vgg_block(2, 128)
blk.initialize()
x = nd.random.uniform(shape=(2, 3, 16, 16))
y = blk(x)
y.shape

(2L, 128L, 8L, 8L)

In [5]:
def vgg_stack(architecture):
    ret = nn.Sequential()
    for (num_convs, channels) in architecture:
        ret.add(vgg_block(num_convs=num_convs, channels=channels))
    return ret

# 定义一个最简单的一个VGG结构，它有8个卷积层，和跟Alexnet一样的3个全连接层

In [6]:
num_outputs = 10
architecture = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
net = nn.Sequential()
with net.name_scope():
    net.add(
        vgg_stack(architecture),
        nn.Flatten(),
        nn.Dense(4096, activation="relu"),
        nn.Dropout(.5),
        nn.Dense(4096, activation="relu"),
        nn.Dropout(.5),
        nn.Dense(num_outputs)
    )

In [7]:
# 训练
import sys
sys.path.append('..')
import utils
from mxnet import gluon
from mxnet import init

train_data, test_data = utils.load_data_fashion_mnist(
    batch_size=64, resize=96)

ctx = utils.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.05})
utils.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=1)

SyntaxError: Non-ASCII character '\xe4' in file utils.py on line 348, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details (utils.py, line 348)